In [3]:
# Make lists of the guest smiles strings and the molecule identifiers 
file = 'CB8_guest_smiles.txt'

file = open(file, 'r')
text = file.readlines()
file.close()

print("Getting list of molecule IDs...")   
MoleculeIDs = [] #SAMPL8 Molecule ID 
smiles = [] #isomeric SMILES

#Loop over lines and parse
for line in text:
    tmp = line.split(';') #Split into columns
    MoleculeID = tmp[1].strip() #mol ID
    smi = tmp[0] #smiles string
    try:
        MoleculeIDs.append(MoleculeID)
        smiles.append(smi)
    except:
        print("Error storing line: %s" % line)
print("Done!")

Getting list of molecule IDs...
Done!


In [4]:
from openeye.oechem import *
from openeye.oeomega import * # conformer generation
from openeye.oequacpac import * 

for idx in range( len( smiles ) ):
    #initialize omega, this is a conformation generator
    omega = OEOmega()
    #set the maximum conformer generated to 1
    omega.SetMaxConfs(1) 
    omega.SetIncludeInput(False)
    omega.SetStrictAtomTypes(False) #Leniency in assigning atom types
    omega.SetStrictStereo(True) #Don't generate conformers if stereochemistry not provided. Setting to false would pick a random stereoisomer
    
    # Generate new OEMol and parse SMILES
    mol = OEMol()
    OEParseSmiles( mol, smiles[idx])
    OESetNeutralpHModel(mol)
    # Generate one conformer
    status = omega(mol)
    if not status:
        print("Error generating conformers for %s, %s. Mol index is %s" % (smiles[idx], MoleculeIDs[idx],idx))

    # Write out PDB of molecule
    ofile = oemolostream('%s.pdb'%(MoleculeIDs[idx]))
    OEWriteMolecule(ofile, mol)
    ofile.close()

    # Write out MOL2 of molecule
    ofile = oemolostream('%s.mol2'%(MoleculeIDs[idx]))
    OEWriteMolecule(ofile, mol)
    ofile.close()
    
    # Write out SDF of molecule
    ofile = oemolostream('%s.sdf'%(MoleculeIDs[idx]))
    OEWriteMolecule(ofile, mol)
    ofile.close()

Error generating conformers for CN(CC1)[C@H]2[C@]3([H])C=C[C@H](O)[C@@]4([H])[C@]31C5=C(O4)C(O)=CC=C5C2, G3. Mol index is 2
Error generating conformers for CN(CC1)[C@H]2[C@]([C@]1([C@@]3([H])O4)C5=C4C(O)=CC=C5C2)([H])CCC3=O, G4. Mol index is 3
Error generating conformers for O=C1CCCCC1(NC)C2=CC=CC=C2Cl, G5. Mol index is 4
Error generating conformers for CN1[C@H]2CC[C@@H]1[C@@H](C(OCC)=O)[C@@H](OC(C3=CC=CC=C3)=O)C2, G7. Mol index is 6


In [5]:
# Make files for O=C1CCCCC1(NC)C2=CC=CC=C2Cl (G5) by picking a random stereoisomer, idx 4 -- this is the racemate
# Do same for G3 (idx 2), G4 (idx 4), G7 (6) -- these come up only when we set to neutral pH
# so it's likely protonation of the nitrogen?
idxs = [2, 3, 4, 6]

omega = OEOmega()
#set the maximum conformer generated to 1
omega.SetMaxConfs(1) 
omega.SetIncludeInput(False)
omega.SetStrictAtomTypes(False) #Leniency in assigning atom types
omega.SetStrictStereo(False) #Don't generate conformers if stereochemistry not provided. Setting to false would pick a random stereoisomer

for idx in idxs:
    # Generate new OEMol and parse SMILES
    mol = OEMol()
    OEParseSmiles( mol, smiles[idx])
    # Generate one conformer
    status = omega(mol)
    if not status:
        print("Error generating conformers for %s, %s. Mol index is %s" % (smiles[idx], MoleculeIDs[idx],idx))

    # Write out PDB of molecule
    ofile = oemolostream('%s.pdb'%(MoleculeIDs[idx]))
    OEWriteMolecule(ofile, mol)
    ofile.close()

    # Write out MOL2 of molecule
    ofile = oemolostream('%s.mol2'%(MoleculeIDs[idx]))
    OEWriteMolecule(ofile, mol)
    ofile.close()

    # Write out SDF of molecule
    ofile = oemolostream('%s.sdf'%(MoleculeIDs[idx]))
    OEWriteMolecule(ofile, mol)
    ofile.close()